In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor

In [2]:
market = Market()
market.connect()
dates = market.retrieve_date_range()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
stuff = pd.to_datetime(dates["date"])

In [4]:
start = (stuff.max() + timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2021-12-08 2021-12-09


In [5]:
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        data = TiingoExtractor.extract(ticker,start,end)
        data["ticker"] = ticker
        market.store("prices",data)
    except Exception as e:
        print(str(e))
market.disconnect()

 16%|█████████████████████▍                                                                                                              | 82/505 [00:13<01:03,  6.71it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


 61%|███████████████████████████████████████████████████████████████████████████████▋                                                   | 307/505 [00:50<00:34,  5.67it/s]

project_market prices documents must be a non-empty list


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 503/505 [01:24<00:00,  7.27it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment
Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [01:24<00:00,  5.94it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment
Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


In [6]:
data

In [7]:
analysis = []
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_data(ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        periods = ["year","quarter","month","week"]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(str(e))
market.disconnect()

 13%|████████████████▉                                                                                                                   | 65/505 [02:04<13:58,  1.91s/it]

'date'


 16%|████████████████████▋                                                                                                               | 79/505 [02:30<13:33,  1.91s/it]

'date'


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 502/505 [16:12<00:05,  1.91s/it]

...

'date'


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 504/505 [16:15<00:01,  1.89s/it]

'date'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [16:17<00:00,  1.94s/it]

'date'


In [8]:
a = pd.DataFrame(analysis).merge(sp5.rename(columns={"Symbol":"ticker"})[["ticker","Security","GICS Sector"]],on="ticker",how="left")
a.to_csv("analysis.csv")

In [9]:
period = "day"
a.sort_values(f"{period}_gain",ascending=False).head(50)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector
341,-0.082596,-0.231014,0.223721,0.086870,0.082008,NCLH,Norwegian Cruise Line Holdings,Consumer Discretionary
85,-0.042198,-0.251821,0.191697,0.050027,0.055135,CCL,Carnival Corporation,Consumer Discretionary
405,0.065752,-0.183689,0.172709,0.043183,0.052066,RCL,Royal Caribbean Group,Consumer Discretionary
279,-0.332010,0.013882,0.139200,0.053620,0.044241,LVS,Las Vegas Sands,Consumer Discretionary
458,0.086957,-0.118621,0.158474,0.028643,0.042387,UAL,United Airlines,Industrials
346,0.458325,0.231384,0.090298,0.047358,0.039003,NVR,NVR,Consumer Discretionary
174,0.429568,0.203699,-0.051219,0.093819,0.036008,ETSY,Etsy,Consumer Discretionary
179,0.312248,0.015313,0.107478,0.038056,0.034940,EXPE,Expedia Group,Consumer Discretionary
66,-0.033875,-0.173346,0.022745,0.047343,0.034784,BIIB,Biogen,Health Care
163,0.343455,0.063947,0.111980,0.066988,0.034107,EW,Edwards Lifesciences,Health Care


In [10]:
a.groupby("GICS Sector").mean().sort_values("day_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Health Care,0.185962,0.005143,0.046261,0.026758,0.009344
Consumer Discretionary,0.305873,0.073869,0.072191,0.025025,0.007873
Real Estate,0.484098,0.090176,0.068320,0.022861,0.007388
Industrials,0.279380,0.062701,0.060794,0.015366,0.004854
Communication Services,0.106923,-0.079038,0.052819,0.008318,0.004726
Materials,0.282644,0.073882,0.055934,0.018685,0.003394
Information Technology,0.268154,0.074905,0.045584,0.031111,0.000944
Energy,0.637096,0.052180,0.069267,0.030054,-0.000674
Utilities,0.137228,0.062028,0.049889,0.005950,-0.000970
